In [1]:
import pandas as pd

from datetime import datetime
from google.cloud import bigquery
from utils import read_json, query_route, add_plate_numbers_to_df, insert_rows_bigquery

BQ_API_KEY = open("credentials/google_bq_key", "r").readlines()[0]
MAPBOX_API_KEY = open("credentials/mapbox_key", "r").readlines()[0]

client = bigquery.Client(project="eco-folder-402813")

# Load in the static dictionaries 
stop_coords_mapping_dict = read_json("data/stop_coords_mapping_dict.json")
jeep_route_mapping_dict = read_json("data/jeep_route_mapping_dict.json")
jeep_information_dict = read_json("data/jeep_information_dict.json")
route_stops_mapping_dict = read_json("data/route_stops_mapping_dict.json")


In [3]:
historical_geocoding_table = pd.DataFrame({
    "names": ["A", "B", "C"],
    "lng": [121.5, 121.25, 121.0],
    "lat": [14.5, 14.25, 14.0]
})

historical_eta_table = pd.DataFrame({
    "stop_id": ["AL1", "AL1", "AL1", "AL2", "AL2"], 
    "lng": [121.98, 121.25, 121.66, 121.23, 121.95], 
    "lat": [14.23, 14.74, 14.63, 14.33, 14.04],
    "time": [datetime.now()]*5, 
    "eta": [5.52, 7.84, 2.25, 11.54, 4.25]
})


In [11]:
use_mapbox = True

temp = query_route(
    route_name = "aurora_loop",
    jeep_route_mapping_dict = jeep_route_mapping_dict, 
    route_stops_mapping_dict = route_stops_mapping_dict, 
    stop_coords_mapping_dict = stop_coords_mapping_dict, 
    historical_geocoding_table = historical_geocoding_table,
    historical_eta_table = historical_eta_table,
    use_mapbox = use_mapbox,
    MAPBOX_API_KEY = MAPBOX_API_KEY
    )
# Retrieve tables and append to historical 
historical_geocoding_table = temp.pop("historical_geocoding_table")
historical_eta_table = temp.pop("historical_eta_table")

/Users/lorenzoflores/Documents/GitHub/jeep-tracking/utils.py:272: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_historical_geocoding_table = pd.concat([new_historical_geocoding_table, new_historical_geocoding_rows], ignore_index=True)
/Users/lorenzoflores/Documents/GitHub/jeep-tracking/utils.py:273: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_historical_eta_table = pd.concat([new_historical_eta_table, new_historical_eta_rows], ignore_index=True)


In [12]:
if use_mapbox:
    insert_rows_bigquery(client, "eco-folder-402813.historical_etas.test", historical_eta_table, row_ids=None)
    insert_rows_bigquery(client, "eco-folder-402813.historical_locations.test", historical_geocoding_table, row_ids=None)


In [13]:
df_temp = pd.DataFrame(temp)
df_temp["jeep_plate_num_list"] = add_plate_numbers_to_df(df_temp["jeep_ids_list"], jeep_information_dict)
df_temp["insertion_time"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

for col in ['jeep_ids_list', 'jeep_locations_list', 'jeep_arrival_times_list', 'jeep_plate_num_list']:
    df_temp[col] = df_temp[col].apply(lambda lst: ",".join([str(x) for x in lst]))

In [18]:
insert_rows_bigquery(client, "eco-folder-402813.jeep_etas.test", df_temp, row_ids=None)

'New rows have been added.'